In [1]:
import torch
import torch.nn as nn
import pickle
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 64
batch_size = 128

max_iters = 5000
learning_rate = 3e-4
eval_iters = 100
dropout = 0.2

n_embd = 384 # vector that is 385 long about certain token
n_head = 8
n_layer = 8 # decoder count
dropout = 0.2 # to prevent overfitting, dropping 20% of neurons

cuda


In [2]:
with open('The_great_gatsby.txt', 'r', encoding='utf-8') as file:
    text = file.read()
chars = sorted(set(text))
vocab_size = len(chars)
print(chars)

['\n', ' ', '!', '#', '$', '%', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ç', 'é', 'ê', 'ô', '\u200a', '—', '‘', '’', '“', '”', '•', '…', '™', '\ufeff']


In [3]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    get = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')

print(x)
print('targets:')
print(y)

inputs:
tensor([[55, 58, 71,  ...,  1, 73, 68],
        [58,  0, 54,  ..., 54, 73, 62],
        [ 1, 30, 60,  ..., 72,  1, 74],
        ...,
        [78,  1, 62,  ..., 72,  1, 76],
        [74, 60,  1,  ..., 67,  0, 58],
        [58, 58, 67,  ..., 62, 64, 58]], device='cuda:0')
targets:
tensor([[58, 71, 60,  ..., 73, 68,  1],
        [ 0, 54, 60,  ..., 73, 62, 68],
        [30, 60, 60,  ...,  1, 74, 67],
        ...,
        [ 1, 62, 67,  ...,  1, 76, 54],
        [60,  1, 55,  ...,  0, 58, 62],
        [58, 67,  1,  ..., 64, 58,  1]], device='cuda:0')


In [5]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for tr in range(block_size):
    context = x[:tr+1]
    target = y[tr]
    print('when', context, 'target', target)

when tensor([93]) target tensor(45)
when tensor([93, 45]) target tensor(61)
when tensor([93, 45, 61]) target tensor(58)
when tensor([93, 45, 61, 58]) target tensor(1)
when tensor([93, 45, 61, 58,  1]) target tensor(41)
when tensor([93, 45, 61, 58,  1, 41]) target tensor(71)
when tensor([93, 45, 61, 58,  1, 41, 71]) target tensor(68)
when tensor([93, 45, 61, 58,  1, 41, 71, 68]) target tensor(63)
when tensor([93, 45, 61, 58,  1, 41, 71, 68, 63]) target tensor(58)
when tensor([93, 45, 61, 58,  1, 41, 71, 68, 63, 58]) target tensor(56)
when tensor([93, 45, 61, 58,  1, 41, 71, 68, 63, 58, 56]) target tensor(73)
when tensor([93, 45, 61, 58,  1, 41, 71, 68, 63, 58, 56, 73]) target tensor(1)
when tensor([93, 45, 61, 58,  1, 41, 71, 68, 63, 58, 56, 73,  1]) target tensor(32)
when tensor([93, 45, 61, 58,  1, 41, 71, 68, 63, 58, 56, 73,  1, 32]) target tensor(74)
when tensor([93, 45, 61, 58,  1, 41, 71, 68, 63, 58, 56, 73,  1, 32, 74]) target tensor(73)
when tensor([93, 45, 61, 58,  1, 41, 71, 6

In [6]:
@torch.no_grad() # good for reporting
def estimate_loss():
    out = {}
    model.eval() # model evaluation, dropout disabled
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # model training mode, dropout enabled
    return out

In [7]:
#initialise neural network
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)]) # for encoder layers

        self.ln_f = nn.LayerNorm(n_embd) # final layer
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean = 0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
            elif isinstance(module,nn.Embedding):
                torch.nn.init.normal_(module.weight, mean = 0.0, std=0.02)

    def forward(self, index, targets=None):
        # logits = self.token_embedding_table(index)
        B, T = index.shape

        tok_emb = self.token_embedding_table(index) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # B batch, T time, C channels (vocab size)
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss= self.forward(index) # getting predictions
            logits = logits[:, -1, :] # becomes (B, C), focis only on last time step
            probs = F.softmax(logits, dim=-1) # softmax function to get probabilities
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            index = torch.cat((index, index_next), dim=1) # (B, T+1), append sampled index to running sequence
        return index

class Block(nn.Module):
    # communication followed by computation
    def __init__(self, n_embd, n_head):
        # n_embd embedding dimensions
        # n_head number of heads
        super().__init__() 
        head_size = n_embd // n_head # division
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x) # self attention
        x = self.ln1(x+y) # add a layer norm
        y = self.ffwd(x) # feed forward
        x = self.ln2(x+y) # add a layer norm
        return x

class FeedForward(nn.Module):
    # linear layer followed by a non-linearity
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),)

    def forward(self, x):
        return self.net(x)

class MultiHeadAttention(nn.Module):
    # multiple heads of self-attention in parallel
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) # runs heads in parallel
        self.proj = nn.Linear(head_size * num_heads, n_embd) # projection to be hackable?, also can add , bias=False
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # concatinate each head together along the last dimention (B, T, F(this one))
        out = self.dropout(self.proj(out))
        return out

class Head(nn.Module):
    # head of self-attention
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False) # transoform head size
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B, T, C = x.shape
        k = self.key(x) # (B, T, hs)
        q = self.query(x)

        # conpute attention scores
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        # weighted aggregation of the values
        v = self.value(x) # (B, T, hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


model = GPTLanguageModel(vocab_size)

print('loading model parameters.')
with open('model_01.pkl', 'wb') as f
    model = pickle.load(f)
print('loaded.')

m = model.to(device)

In [8]:
#optimizes
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']}, val loss: {losses['val']}")
    
    xb, yb = get_batch('train') # sample batch of data
    logits, loss = model.forward(xb, yb) #evaluate the loss
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    #print(loss.item())

with open('model_01.pkl', 'wb') as f
    pickle.dump(model, f)
print('model saved.')

step: 0, train loss: 4.6487836837768555, val loss: 4.647613048553467
step: 100, train loss: 2.342151403427124, val loss: 2.340352773666382
step: 200, train loss: 2.037874221801758, val loss: 2.0387651920318604
step: 300, train loss: 1.862667202949524, val loss: 1.8642381429672241
step: 400, train loss: 1.7432917356491089, val loss: 1.7405171394348145
step: 500, train loss: 1.6528804302215576, val loss: 1.658371090888977
step: 600, train loss: 1.5829946994781494, val loss: 1.5836008787155151
step: 700, train loss: 1.5250871181488037, val loss: 1.527219533920288
step: 800, train loss: 1.4830257892608643, val loss: 1.4844839572906494
step: 900, train loss: 1.4421733617782593, val loss: 1.438604712486267
step: 1000, train loss: 1.4036064147949219, val loss: 1.402845859527588
step: 1100, train loss: 1.3683358430862427, val loss: 1.3646725416183472
step: 1200, train loss: 1.3418959379196167, val loss: 1.3440251350402832
step: 1300, train loss: 1.3124696016311646, val loss: 1.3121479749679565

In [9]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)

RuntimeError: The size of tensor a (64) must match the size of tensor b (65) at non-singleton dimension 2